# Data Structure

- $i$: industries, 1-5
- $t$: regions, 1-7
- Each single observation is indexed by $(i,t)$:
$$
x_{i,t} = x_i + x_t + e_{i,t}
$$

# Test Statistic

First, we generate the matrix $R$ defined in equation (1) of the paper from random values. We go with some random data. 

In [9]:
import importlib

if importlib.util.find_spec('numpy') is None:
    !pip3 install numpy

if importlib.util.find_spec('scipy') is None:
    !pip3 install scipy

import numpy as np
from scipy.stats import chi2

In [10]:
p = 5
k = 7
n = 1

f_sq = 0
for i in range(n):
    Vi = np.random.rand(p, 1)
    Zi = np.random.rand(k, 1)
    fi = np.kron(Vi,Zi) # 35*1
    f_sq += np.dot(fi,fi.T) # 35*35

R = f_sq/n # 35*35

Then we stack $R$ to $\mathcal{R}(R)$ defined in equation (5).

In [11]:
R_cal = np.empty([p ** 2, k ** 2])
for i in range(1,p):
    for j in range(1,p):
        block_ij = R[((i-1)*k):(i*k), ((j-1)*k):(j*k)] # k*k     
        vector_ij = block_ij.flatten(order='F') # k^2*1
        R_cal[(j-1)*p+i-1,:] = vector_ij # the p-th row

from which we can perform the singular value decomposition (equation (9) in the paper):

In [12]:
L, Sigma_v, Nt = np.linalg.svd(R_cal)
Sigma = np.diag(Sigma_v)
N = Nt.T

length = max(np.shape(L),np.shape(N))[0]
width = min(np.shape(L),np.shape(N))[0]
diff = length - width 

happend = np.zeros([diff+1, diff])
vappend = np.zeros([diff, diff+1])

if np.shape(L)[0] > np.shape(N)[0]:
  Sigma = np.vstack((Sigma, vappend))
elif np.shape(L)[0] < np.shape(N)[0]:
  Sigma = np.hstack((Sigma, happend))

and obtain the components of the KPST

In [13]:
L2 = np.delete(L, 0, 1)
Sigma2 = np.delete(Sigma, 0, 1)
Sigma2 = np.delete(Sigma2, 0, 0)
N2 = np.delete(N, 0, 1)

vec_R_cal = R_cal.flatten(order='F')
V = np.outer(vec_R_cal.T,vec_R_cal.T)

Now we can calculate KPST based on equation (22):

In [14]:
K1 = Sigma2.flatten(order='F') 

K2 = np.kron(N2,L2).T@V@np.kron(N2,L2)

KPST = n* K1.T @np.linalg.inv(K2) @K1

# Example

Now we have the KPST statistic in hand, we compare it with $\chi^2(df,1-\alpha)$, where the degree of freedom $df$ is calculated using equation (25).

In [15]:
## Put all the previous steps in a function

def kpst(p=5,k=7,n=30):
   
   f_sq = 0
   
   for i in range(n):
      Vi = np.random.rand(p, 1)
      Zi = np.random.rand(k, 1)
      fi = np.kron(Vi,Zi) # 35*1
      f_sq += np.dot(fi,fi.T) # 35*35
  
   R = f_sq/n # 35*35
   
   R_cal = np.empty([p ** 2, k ** 2])
   
   for i in range(1,p):
    for j in range(1,p):
        block_ij = R[((i-1)*k):(i*k), ((j-1)*k):(j*k)] # k*k     
        vector_ij = block_ij.flatten(order='F') # k^2*1
        R_cal[(j-1)*p+i-1,:] = vector_ij # the p-th row
        
   L, Sigma_v, Nt = np.linalg.svd(R_cal)
   Sigma = np.diag(Sigma_v)
   N = Nt.T
   
   length = max(np.shape(L),np.shape(N))[0]
   width = min(np.shape(L),np.shape(N))[0]
   diff = length - width 
   
   happend = np.zeros([diff+1, diff])
   vappend = np.zeros([diff, diff+1])
   
   if np.shape(L)[0] > np.shape(N)[0]:
    Sigma = np.vstack((Sigma, vappend))
   elif np.shape(L)[0] < np.shape(N)[0]:
    Sigma = np.hstack((Sigma, happend))
  
   L2 = np.delete(L, 0, 1)
   Sigma2 = np.delete(Sigma, 0, 1)
   Sigma2 = np.delete(Sigma2, 0, 0)
   N2 = np.delete(N, 0, 1)
   vec_R_cal = R_cal.flatten(order='F')
   V = np.outer(vec_R_cal.T,vec_R_cal.T)

   K1 = Sigma2.flatten(order='F') 
   K2 = np.kron(N2,L2).T@V@np.kron(N2,L2)
   KPST = n* K1.T @np.linalg.inv(K2) @K1
   return KPST

## Function that returns the degree of freedom
def dof(p,k):
  dof = (0.5*k*(k+1)-1)*(0.5*p*(p+1)-1)
  return dof

Let's try performing the test:

In [16]:
kpst_obs = []

## Use a loop to deal with the case of not-inversible matrix
while len(kpst_obs) < 1:
    new_value = kpst(p=5,k=7,n=30)
    if ~np.isnan(new_value):
        kpst_obs.append(new_value) 

## But there is still possibility of non-convergence of SVD, just rerun the code on error

## Test
if kpst_obs[0] < chi2.ppf(1-.05, dof(p=5,k=7)):
    print("We can't reject the null that R has KPS at nominal size 0.05. We conclude that R has KPS.")
else: 
    print("We reject the null that R has KPS at nominal size 0.05. We conclude that R doesn't has KPS.")

We can't reject the null that R has KPS at nominal size 0.05. We conclude that R has KPS.
